In [ ]:
import seaborn as sns 
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
data = pd.read_csv('../data/Titanic.csv')
data.head()

In [ ]:
print('All objects', data.shape[0])
for col in data.columns:
    print('{:11} {:3}'.format(col, data[col].nunique()))

In [ ]:
data.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
cat_cols = [col for col in data.columns if data[col].dtype == 'object']

In [ ]:
data[cat_cols] = data[cat_cols].fillna('NoneType')

In [ ]:
for col in cat_cols:
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])

In [ ]:
X, y = data.drop('Survived', axis=1), data['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    test_size=0.2, random_state=777) 

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
X_train = X_train.fillna(X_train.mean()).astype(float)
X_test = X_test.fillna(X_test.mean()).astype(float)

In [ ]:
ms = MinMaxScaler()
ms.fit(X_train)
X_train = pd.DataFrame(ms.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(ms.transform(X_test), columns=X_test.columns)
#Здесь не обязательно пересоздавать датафрейм, но пока данные маленькие - так нагляднее.

In [ ]:
X_train.head()

Добейтесь наилучшего качества (accuracy) на тестовой выборке

In [ ]:
base_clf = KNeighborsClassifier() # инициализируем базовый классификатор
cv = KFold(n_splits=3, shuffle=True, random_state=42)

params_grid = {
    'n_neighbors': list(range(1, 50, 2)), # Перебираем от 1 до 50 с шагом 2
    'weights': ['uniform', 'distance'], # с каким весом берём каждого соседа. uniform - с одинаковым, distance - обратно пропорционально расстоянию. Ещё можно передать свою функцию
    'algorithm': ['auto'], # алгоритм поиска ближайших соседей. Влияет только на скорость обучения
    'leaf_size': [30], # параметры некоторых алгоритмов поиска ближ соседей. Влияет только на скорость обучения
    'metric': ['minkowski'], # метрика расстояния, используемая в алгоритме поиска ближайших соседей
    'p': [2] # параметр метрики, используемой в методе ближайших соседей 
}
gs = GridSearchCV(base_clf, params_grid, cv=cv, scoring='accuracy',
                  return_train_score=True) # cv - количество частей (K из рис. выше), verbose - выводить параметры во время обучения

gs.fit(X_train, y_train)

In [ ]:
gs.best_score_ 

In [ ]:
gs.best_params_

In [ ]:
clf = gs.best_estimator_
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
n = 1
means = []
for split in list(zip(gs.cv_results_['split0_test_score'], 
         gs.cv_results_['split1_test_score'], 
         gs.cv_results_['split2_test_score'])): 
    means.append(np.mean(split))
    print('{:2}. mean: {:.3f} std: {:.3f}'.format(n, np.mean(split), np.std(split)))
    n+=1

In [ ]:
plt.plot(means)